In [1]:
# Download prerequisites
%pip install transformers

Note: you may need to restart the kernel to use updated packages.


DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\filelock-3.13.1-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\fsspec-2023.10.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish\appdata\local\programs\python\python311\lib\site-packages\mpmath-1.3.0-py3.11.egg is deprecated. pip 24.3 will enforce this behaviour change. A possible replacement is to use pip for package installation.. Discussion can be found at https://github.com/pypa/pip/issues/12330
DEPRECATION: Loading egg at c:\users\irish

In [2]:
# Import the needed libraries
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
import numpy as np
import pandas as pd

In [3]:
# Load a pre-trained LLM and tokenizer
model_name = "gpt2" # Replace with name of the LLM you want to run
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

model.safetensors:  21%|##1       | 115M/548M [00:00<?, ?B/s]

Error while downloading from https://cdn-lfs.huggingface.co/gpt2/248dfc3911869ec493c76e65bf2fcf7f615828b0254c12b473182f0f81d3a707?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1722063686&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjA2MzY4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9ncHQyLzI0OGRmYzM5MTE4NjllYzQ5M2M3NmU2NWJmMmZjZjdmNjE1ODI4YjAyNTRjMTJiNDczMTgyZjBmODFkM2E3MDc%7EcmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=kEn4B%7EtNJWyGmSS1IK58Mjgai%7E%7EChousDPq6H%7EkFB%7EFggBW9qsUGKeuXeecNW9vADdF4SejfNscQjSkQwMTivTeqFL%7EC8ogoOLvZw2yJF62oWAlNPuue9dcvgC7w06qnV8kt1mDA26SpKWtFDo5RIaXiHPqSoux49tbi-ZRL5iNBGYny-UuNYreY0GNtRIJbUQdrNqfAbkbfrMKFk3TcApJRYssgD2OqtezDvCnrfS%7EWwfgdWBuUqUboiFCunDuHDca07NizXuhM74I5OrWGTFqZ6toFIc0APUGUo6Cy1IW-V1FnWn8x7LjNxJOZPNAiQnNMNlkzBu3ZWqeOqan6lw__&Key-Pair-Id=K3ESJI6DHPFC7: HTTPSConnectionPool(host='cd

ConnectTimeout: (MaxRetryError("HTTPSConnectionPool(host='cdn-lfs.huggingface.co', port=443): Max retries exceeded with url: /gpt2/248dfc3911869ec493c76e65bf2fcf7f615828b0254c12b473182f0f81d3a707?response-content-disposition=inline%3B+filename*%3DUTF-8%27%27model.safetensors%3B+filename%3D%22model.safetensors%22%3B&Expires=1722063686&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcyMjA2MzY4Nn19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy5odWdnaW5nZmFjZS5jby9ncHQyLzI0OGRmYzM5MTE4NjllYzQ5M2M3NmU2NWJmMmZjZjdmNjE1ODI4YjAyNTRjMTJiNDczMTgyZjBmODFkM2E3MDc~cmVzcG9uc2UtY29udGVudC1kaXNwb3NpdGlvbj0qIn1dfQ__&Signature=kEn4B~tNJWyGmSS1IK58Mjgai~~ChousDPq6H~kFB~FggBW9qsUGKeuXeecNW9vADdF4SejfNscQjSkQwMTivTeqFL~C8ogoOLvZw2yJF62oWAlNPuue9dcvgC7w06qnV8kt1mDA26SpKWtFDo5RIaXiHPqSoux49tbi-ZRL5iNBGYny-UuNYreY0GNtRIJbUQdrNqfAbkbfrMKFk3TcApJRYssgD2OqtezDvCnrfS~WwfgdWBuUqUboiFCunDuHDca07NizXuhM74I5OrWGTFqZ6toFIc0APUGUo6Cy1IW-V1FnWn8x7LjNxJOZPNAiQnNMNlkzBu3ZWqeOqan6lw__&Key-P

In [ ]:
# Define the device to run the model (GPU or CPU)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Creating the Dataset

This generates mock data for a dataset of essays on which to fine-tune a local model. Essays are generated with Google Gemini (hehe), and essay scores/targets are manually created.

In [ ]:
columns = ['essay_set', 'essay', 'ground_truth_score']
data = [
    [
        1,
        "Is everyone freaking out about computers for no reason? Computers are awesome! They're like having a superhero sidekick in your pocket. They can teach you anything you want to know, from how to build a volcano (for science class, of course!) to what life is like on the other side of the world. Plus, games help your hand-eye coordination, which is basically a superpower for gamers like me.\n\nSure, maybe some grown-ups spend too much time staring at screens, but that's their problem, not ours. We can totally use computers for good stuff and still hang out with friends or play outside.  So, lighten up everyone, computers are here to stay, and they're making our lives way cooler!\n\nYour friend,\n\nGamer Girl extraordinaire",
        2
    ],
    [
        1,
        "Dear Editor,\n\nComputers are like a double-edged sword. On one hand, they're amazing tools for learning and connecting. I can chat with my cousin in California even though she's miles away, and research projects are a breeze with all the information online. Plus, there are these awesome educational games that make learning history or even another language actually fun!\n\nBut on the other hand, sometimes computers can be a total time suck. You start checking one thing, and then BAM! Two hours later, you're watching cat videos (don't judge, they're hilarious!).  This can lead to neglecting real-life stuff like hanging out with friends or getting enough exercise.\n\nSo, I think the key is balance. Computers are great, but we gotta make sure they don't take over our lives. Maybe parents can set screen time limits, and we can all make a conscious effort to spend more time outside and with loved ones.\n\nSincerely,\n\nA Balanced Computer User",
        4
    ],
    [
        1,
        "Dear Editor,\n\nSome people say computers are turning us all into couch potatoes. They might have a point. It's way easier to watch funny videos online than go for a bike ride.  But computers can actually help you appreciate nature too! You can research cool animals or amazing places you want to visit someday.  Plus, there are apps that help you identify plants and track your hikes.\n\nThink of it like this: computers are like a superhighway to information, but sometimes you gotta take an exit and explore the real world. We can use computers to learn about nature, then go outside and experience it for ourselves. It's all about finding the right balance!\n\nYours truly,\n\nThe Tech-Savvy Nature Enthusiast",
        3
    ],
    [
        1,
        "Dear Editor,\n\nComputers aren't just about games and social media, you know! They can be a real spark for creativity.  I can write stories on the computer, way easier than using pen and paper. Plus, there are these amazing programs that let you design animations or even create your own music!  Imagine composing a song as cool as your favourite band, all on your computer.\n\nSure, some folks might say computers make us lazy, but I think they actually help us think outside the box. You can find inspiration for anything online, from historical fashion to crazy science experiments.  Computers are like a giant toolbox full of cool stuff to help you express yourself.\n\nSo next time someone tells you to get off the computer, tell them you're busy building your own creativity empire!\n\nYours creatively,\n\nThe Techie Da Vinci",
        4
    ],
    [
        1,
        "Dear Editor,\n\nLet's be honest, computers can be way better friends than some actual people. They're always there, 24/7, whenever you need them. Feeling bored? Boom, there's a million games to play. Need help with homework? Online resources can explain anything. Plus, you can connect with people who share your interests, no matter how weird or wonderful they are.\n\nOf course, computers can't replace real-life friends entirely. But they can be a great way to meet new people, especially if you're shy or new to town.  Think of them as awesome companions who can help you learn, play, and connect with the world.\n\nYour friend (both online and off),\n\nThe Social Butterfly with a Keyboard",
        3
    ],
    [
        1,
        "Dear Editor,\n\nFor history buffs like me, computers are a dream come true!  Imagine having access to museums and libraries from all over the world, all at your fingertips. You can explore ancient civilizations, watch historical battles come to life in documentaries, or even take virtual tours of famous landmarks.\n\nTextbooks are cool, but computers let you experience history in a whole new way. You can see real photos and videos, listen to speeches from famous people, and even read primary source documents.  It's like stepping right back in time, without needing a time machine (although that would be pretty awesome too!).\n\nSo next time you hear someone complaining about computers, tell them about the history buff's paradise they're missing out on!\n\nSincerely,\n\nThe Time-Traveling Student (via Computer)",
        3
    ],
    [
        1,
        "Dear Editor,\n\nThe conversation about computers seems stuck in a binary – productivity boosters or social isolation machines. While I agree computers offer undeniable benefits – research efficiency, global communication, and even entertainment – I believe their impact on productivity might be a double-edged sword.\n\nOn the surface, computers streamline tasks, from research to collaboration. Gone are the days of scouring libraries; now, a quick keyword search yields mountains of information. However, this very convenience can be a productivity trap. The endless stream of notifications, social media updates, and the allure of online entertainment can easily derail focus.\n\nFurthermore, the multitasking culture computers foster might be an illusion. Studies suggest rapid switching between tasks actually diminishes efficiency. The constant barrage of stimuli creates a fear of missing out mentality, leading us to flit between tasks, achieving none in depth.\n\nSo, how do we utilize the benefits of computers without succumbing to their productivity paradox? Perhaps a conscious effort towards focused work sessions, utilizing time management tools, and even strategically disconnecting from the digital world could be the answer.\n\nComputers are powerful tools, but ultimately, it's up to us to harness their potential for true productivity, not get lost in the digital whirlwind.\n\nSincerely,\n\nA Student in the Age of Distraction",
        5
    ],
    [
        1,
        "Dear Editor,\n\nThe internet age, ushered in by computers, has undeniably expanded our access to information and connection. However, I believe the rise of \"echo chambers\" on social media platforms presents a significant threat to critical thinking and genuine discourse.\n\nComputers have created personalized online experiences. News feeds curate content based on our preferences, surrounding us with information that confirms our existing beliefs. This insular environment fosters confirmation bias, where we dismiss opposing viewpoints and cling to opinions that resonate with our own.\n\nThis lack of exposure to diverse perspectives hinders intellectual growth and stifles healthy debate.  Civil discourse necessitates grappling with opposing views, challenging our assumptions, and fostering empathy. Echo chambers, on the other hand, create intellectual bubbles, leading to polarization and a decline in open-mindedness.\n\nTo counteract these effects, we must consciously seek out diverse viewpoints on computers themselves. We can utilize search engines to find information from various sources and engage in respectful online discussions with those who hold differing opinions.\n\nComputers are powerful tools for connection, but they shouldn't limit our intellectual horizons.  Let's harness their potential to create a more informed and intellectually diverse society.\n\nSincerely,\n\nA Student Who Values Open Discourse",
        5
    ],
    [
        1,
        "Dear Editor,\n\nThe internet, facilitated by computers, has democratized access to information and ignited a wave of social activism. However, the ease of online engagement might be creating a culture of \"clicktivism\" – shallow, performative activism that prioritizes social media validation over meaningful action.\n\nComputers allow us to instantly share petitions, raise awareness for causes, and connect with like-minded individuals. However, this ease of engagement can lead to a sense of accomplishment without actual effort. Signing a petition online might feel good, but it doesn't translate to real-world change unless followed by concrete action.\n\nFurthermore, the curated nature of social media feeds can create a skewed perception of reality. We might be bombarded with success stories, fostering a sense of discouragement when our own efforts seem insignificant. This can lead to slacktivism, where individuals feel absolved of their responsibility after a simple click.\n\nComputers can be powerful tools for social change, but we must guard against mistaking online engagement for genuine activism. It's crucial to translate our digital outrage into concrete action, volunteering, and creating a dialogue with those who hold opposing views.  Only then can we  leverage the power of computers to create lasting social change.\n\nSincerely,\n\nA Student Who Believes in Action, Not Just Likes",
        5
    ],
    [
        1,
        "Dear Editor,\n\nComputers are like double-stuffed Oreos: awesome and delicious, but you gotta be careful not to overdo it. Sure, they can be educational – research projects are a breeze with all that info online, and some games are actually pretty challenging (think reflexes of a ninja!). Plus, you can video chat with your bestie across town, way cooler than a boring phone call.\n\nBut let's face it, computers can be a major time suck. You hop on to check something quick, and next thing you know, hours have flown by scrolling through social media or watching random videos.  Suddenly, you haven't gotten any homework done, and your mom's nagging you to get outside.\n\nThe key is balance, people! Computers are great tools, but they shouldn't replace real life. We need to get our exercise, spend time with friends and fam, and maybe even appreciate some fresh air (believe it or not, the world outside your screen is pretty cool too!). So, let's use computers to our advantage, but remember, there's a whole world out there waiting to be explored –  the non-digital kind!\n\nSincerely,\n\nThe Tech-Savvy (But Not Screen-Obsessed) Teenager",
        3
    ],
    [
        1,
        "Dear Editor,\n\nLook, I get it. Some adults think computers are turning us all into antisocial code monkeys. They whine about us glued to screens, neglecting exercise and real-life interactions. But here's the thing: computers can actually be social tools!\n\nThink about it. We can connect with friends online, share jokes and stories, even play games together virtually. Plus, social media lets you meet new people who share your interests, no matter how niche they are. It's like having a giant club for everything under the sun, all online!\n\nSure, computers shouldn't replace face-to-face interactions. But honestly, some people are just way more comfortable expressing themselves online. Plus, there are plenty of introverts and shy kids out there who benefit from the social connection computers offer.\n\nSo, next time you see a teenager staring at a screen, don't assume the worst. We might be having the best virtual hangout ever, making new friends, or even learning a new skill. Computers can be social butterflies too!\n\nYours truly,\n\nThe Socially Connected Screenager",
        3
    ],
    [
        1,
        "Dear Editor,\n\nComputers aren't just a trend, they're the future! They're changing the world, and it's not all doom and gloom like some grumpy grown-ups think.  Learning? Computers make it interactive and fun. Need to connect with someone across the globe? Done in seconds.  Plus, with online courses and coding programs, teenagers like me can actually learn future-proof skills.\n\nSure, too much screen time can be bad, but guess what? We're smart enough to find a balance.  We can use computers to learn about fitness and healthy habits. Plus, there are apps for tracking hikes and even virtual reality experiences that take you to exotic locations.\n\nHonestly, computers are opening doors to a whole new world. We can be part of the solution, not the problem. Let's embrace technology and use it to our advantage. The future is digital, and  guess what? We're ready for it!\n\nSincerely,\n\nThe Tech-Ready Teenager",
        3
    ],
    [
        1,
        "Dear Editor,\n\nThe rise of computers has sparked a heated debate – are they a boon or a bane for society? While some laud their educational and social advantages, concerns about human connection and physical well-being loom large.\n\nUndeniably, computers offer unparalleled access to information. From research for academic papers to virtual tours of the Colosseum, knowledge is democratized. Additionally, online platforms foster social connection – video calls bridge geographical distances, and online communities allow people with shared interests to connect.\n\nHowever, these benefits come at a cost. The allure of the digital world can foster social isolation. Face-to-face interactions, crucial for emotional well-being and developing social skills, can be neglected. Furthermore, the sedentary nature of computer use contributes to a decline in physical activity, leading to health problems.\n\nThe solution lies not in abandoning technology, but in cultivating a mindful relationship with it. We must prioritize real-world interactions and physical activities. Setting time limits for computer use and scheduling outdoor adventures are essential.\n\nUltimately, computers are tools – powerful and transformative, but ultimately neutral. It is up to us to harness their potential while remaining grounded in the real world. Let us not become slaves to technology, but rather its responsible stewards.\n\nSincerely,\n\nA Concerned Citizen of the Digital Age",
        5
    ],
    [
        1,
        "Dear Editor,\n\nThe pervasiveness of computers in our lives presents a double-edged sword – a vast ocean of information at our fingertips, coupled with the potential for intellectual stagnation. While proponents highlight the educational advantages of computers, the ease of access can breed a culture of passive information consumption.\n\nUndoubtedly, computers are gateways to knowledge. Research for projects becomes a breeze, and online courses offer unprecedented educational opportunities. Additionally, global news platforms allow us to stay informed about international affairs.\n\nHowever, the sheer volume of information available online can be overwhelming. The ability to discern credible sources from fake news is crucial, yet neglected. Furthermore, the reliance on pre-summarized information can hinder the development of critical thinking skills.\n\nTo mitigate these risks, we need to foster an environment that encourages active information processing. Evaluating sources, questioning assumptions, and drawing reasoned conclusions are paramount. Additionally, we must advocate for educational reforms that emphasize critical thinking alongside access to information.\n\nIn conclusion, computers can be powerful tools for learning, but only if we cultivate the skills to navigate the information landscape effectively. Let us not become passive consumers of data, but rather discerning learners who utilize technology to expand our intellectual horizons.\n\nSincerely,\n\nA Student of the Digital Age",
        5
    ],
    [
        1,
        "Dear Editor,\n\nThe rise of computers has coincided with a growing concern - the erosion of privacy in the digital age. While proponents highlight the social and educational benefits of computers, the trade-off is often our personal data becoming a valuable commodity for corporations.\n\nUndeniably, computers facilitate social interactions and educational pursuits. Online platforms connect us with loved ones across the globe, and online courses offer flexible learning opportunities.\n\nHowever, our online activities generate a vast trail of data – search history, social media interactions, even location tracking. This data is often collected and monetized by corporations, potentially leading to targeted advertising and even manipulation.\n\nThe solution lies in advocating for stricter data protection laws and fostering a culture of digital hygiene. We must be vigilant about the information we share online and utilize tools to protect our privacy. Additionally, promoting digital literacy is crucial to empower individuals to navigate the complexities of online data collection.\n\nIn conclusion, computers offer undeniable benefits, but at a cost – our privacy. Striking a balance between technological advancement and individual autonomy is paramount. Let us reclaim control over our digital footprints and ensure that computers are tools for empowerment, not instruments of surveillance.\n\nSincerely,\n\nA Concerned Citizen of the Digital Age",
        5
    ],
    [
        1,
        "Dear Editor,\n\nSome grown-ups might moan about computers turning our brains to mush, but they're totally wrong! Computers are like a giant learning playground, way cooler than dusty old textbooks. Wanna learn about the pyramids? Boom, there's a virtual tour online. Need help with a science project? There are tons of websites with experiments you can actually do at home (with adult supervision, of course!).\n\nPlus, forget boring lectures – some online courses have games and quizzes that make learning actually fun! And don't even get me started on coding. It's like learning a secret language that lets you build your own computer games!\n\nSure, sometimes you can get sucked into a black hole of cat videos, but that's why we need balance. Maybe parents can set screen time limits, and we can all make an effort to play outside more. But honestly, computers are here to stay, and they're making learning way more awesome!\n\nYour fellow student,\n\nThe Tech-Savvy Brainiac",
        3
    ],
    [
        1,
        "Dear Editor,\n\nComputers are awesome for some things, but they can also be a bit of a double-edged sword, especially when it comes to friends. On the one hand, they let you connect with people from all over the world, which is pretty cool. You can video chat with your cousin in Hawaii or chat with someone who shares your love for obscure cartoon characters.\n\nBut here's the thing: computers can't replace real-life friends. Sure, you can text your bestie all day, but there's nothing like hanging out in person, cracking jokes, and maybe even getting into a friendly game of dodgeball.\n\nSometimes, computers can make us forget about the people right next to us. We get so caught up in the online world that we forget to spend quality time with our family or even just play with the dog.\n\nSo, the key is balance. Let's use computers to connect with people far away, but don't forget to nurture the friendships we have in real life too. Remember, a computer friend can't give you a high five!\n\nSincerely,\n\nThe Socially Balanced Student",
        4
    ],
    [
        1,
        "Dear Editor,\n\nSome folks think computers are turning us into couch potatoes, glued to screens and forgetting about the great outdoors. They might have a point – who needs a walk in the park when you can explore a rainforest virtually?\n\nBut here's the secret: computers can actually help you appreciate nature even more! You can research cool animals you want to see someday, like majestic tigers or playful dolphins. Plus, there are apps that help you identify plants and track your hikes. Imagine hiking a mountain and using an app to learn all about the different trees you see!\n\nThink of computers as an escape hatch from the everyday. They can transport you to amazing places, but they can also inspire you to experience the real thing. So next time you're feeling cooped up, use your computer to plan an awesome outdoor adventure, not just another virtual one!\n\nYours truly,\n\nThe Tech-Powered Nature Enthusiast",
        4
    ],
    [
        1,
        "Dear Editor,\n\nThe rise of computers has fundamentally reshaped the landscape of knowledge acquisition. While concerns about attention spans and social isolation are valid, the democratization of information access outweighs these anxieties.\n\nPrior to the digital age, knowledge was largely siloed within academic institutions and specialized libraries. Today, computers provide unprecedented access to a vast ocean of information. Online databases, academic journals, and educational platforms empower individuals to pursue intellectual curiosity regardless of geographic location or socioeconomic background.\n\nHowever, the sheer volume of information can be overwhelming. Critical thinking skills are paramount in navigating the digital terrain, discerning credible sources from misinformation. Universities and educators must adapt, fostering information literacy alongside access.\n\nFurthermore, the always-on nature of technology can lead to social isolation and hinder physical well-being.  Encouraging digital  detoxification and promoting healthy screen time habits are crucial. Yet, the ability to connect with colleagues and friends globally fosters a sense of global citizenship and facilitates collaboration across borders.\n\nIn conclusion, the democratizing potential of computers outweighs concerns about their downsides. By fostering critical thinking skills and promoting balance in our digital lives, we can harness the power of technology to empower individuals and advance human knowledge.\n\nSincerely,\n\n",
        5
    ],
    [
        1,
        "Dear Editor,\n\nWhile computers hold immense potential for learning and connection, they can exacerbate existing societal inequalities.  While proponents tout the democratization of information, the reality is a widening algorithmic divide.\n\nUnequal access to computers and reliable internet connections limits opportunities for those in underserved communities.  Without this access, educational pursuits, professional development, and even basic communication are hindered.   Furthermore, the digital divide disproportionately affects marginalized groups, further perpetuating social and economic disparities.\n\nMoreover, the internet thrives on data, leading to algorithmic biases.  Search engines and social media platforms often prioritize content that confirms existing beliefs, creating echo chambers that hinder exposure to diverse viewpoints and critical thinking.\n\nTo mitigate these issues, we need robust investment in public libraries, affordable internet access programs, and digital literacy initiatives. Additionally, platforms need to be held accountable for algorithmic bias, ensuring inclusive and equitable information access.\n\nIn conclusion, the benefits of computers cannot be fully realized without addressing the digital divide and algorithmic bias.  Otherwise, these technological advancements will only exacerbate existing inequalities.  Let us bridge this gap and ensure that computers empower all individuals, not just a privileged few.\n\nSincerely,\n\nA Social Scientist of the Digital Age",
        5
    ],
]

In [ ]:
# Generate essay IDs
start_id = 7085
essay_df = pd.DataFrame(data=data, columns=columns)
essay_df['essay_id'] = np.arange(start_id, start_id + len(data))

In [ ]:
essay_df.head()

In [ ]:
essay_df.to_csv('finetune_set.csv', index=False)

### Train-Test Split

In [ ]:
from pathlib import Path

def read_split(split_dir):
    split_dir = Path(split_dir)
    texts = []
    labels = []
    for label_dir in ["pos", "neg"]:
        for text_file in (split_dir/label_dir).iterdir():
            texts.append(text_file.read_text())
            labels.append(0 if label_dir is "neg" else 1)

    return texts, labels

train_texts, train_labels = read_imdb_split('aclImdb/train')
test_texts, test_labels = read_imdb_split('aclImdb/test')

### Finetuning

In [ ]:
input_text = "Once upon a time"
input_tokens = tokenizer(input_text, return_tensors="pt").to(device)

In [ ]:
with torch.no_grad(): # Tells PyTorch not to train the model on your input
    output_tokens = model.generate(input_tokens ["input_ids"], num_return_sequences=1)

In [ ]:
generated_text = tokenizer.decode(output_tokens [0], skip_special_tokens=True)
print(generated_text)